In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import keras
import cv2
from tensorflow.keras import datasets,layers,models
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle

In [3]:
X_train=np.load("total_features.npy")
y_train=np.load("y_final_train.npy")
X_train_cval=np.load("total_features_cval.npy")
y_cval=np.load("y_final_cval.npy")

In [26]:
X_train.shape

(38400, 80)

In [28]:
X_train_cval.shape

(9600, 80)

In [5]:
X_train.shape

(38400, 80)

In [29]:
X_train.shape[1]

80

In [8]:
from keras.models import Sequential

In [10]:
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation


In [14]:
from keras.layers import normalization

In [38]:
from tensorflow.keras.layers import BatchNormalization

In [39]:
final_model = Sequential()
final_model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
final_model.add(Dropout(0.50))
final_model.add(BatchNormalization())
final_model.add(Dense(256, activation='relu'))
final_model.add(Dropout(0.50))
final_model.add(Dense(16, activation='softmax'))

In [40]:
final_model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [42]:
final_model.fit(X_train,y_train,epochs=50, validation_data=(X_train_cval,y_cval))

Epoch 1/50
1200/1200 [==============================] - 6s 5ms/step - loss: 0.3919 - accuracy: 0.8878 - val_loss: 1.2693 - val_accuracy: 0.7044
Epoch 2/50
1200/1200 [==============================] - 6s 5ms/step - loss: 0.3870 - accuracy: 0.8883 - val_loss: 1.2689 - val_accuracy: 0.7048
Epoch 3/50
1200/1200 [==============================] - 5s 5ms/step - loss: 0.3855 - accuracy: 0.8878 - val_loss: 1.3120 - val_accuracy: 0.7030
Epoch 4/50
1200/1200 [==============================] - 6s 5ms/step - loss: 0.3810 - accuracy: 0.8883 - val_loss: 1.3265 - val_accuracy: 0.7002
Epoch 5/50
1200/1200 [==============================] - 6s 5ms/step - loss: 0.3786 - accuracy: 0.8877 - val_loss: 1.3494 - val_accuracy: 0.7034
Epoch 6/50
1200/1200 [==============================] - 6s 5ms/step - loss: 0.3737 - accuracy: 0.8909 - val_loss: 1.3292 - val_accuracy: 0.7036
Epoch 7/50
1200/1200 [==============================] - 6s 5ms/step - loss: 0.3748 - accuracy: 0.8898 - val_loss: 1.3319 - val_accuracy:

In [44]:
new_path="C:/Users/nithi/.jupyter/new_validation"

In [45]:
import pandas as pd
data = range(17801,18701)
df = pd.DataFrame(data,columns = ['id'])

In [46]:
X_test = []
for i in range(len(df)):
    path = os.path.join(new_path,str(df.iloc[i,0]))
    path = path.replace("\\","/")
    path = path+".tif"
    path=path.replace("\new","/new")
    #path = os.path.join(validation_dir,str(df.iloc[i,0]))
    #path = path.replace("\\","/")
    #path = path + ".tif"
    img = cv2.imread(path)
    img = img / 255
    resized_img = cv2.resize(img,(128,128))
    X_test.append(resized_img)

In [47]:
X_test = np.array(X_test)

In [48]:
model_header=tf.keras.models.load_model("HEADER_MODEL.h5")
model_footer=tf.keras.models.load_model("BOTTOM_MODEL.h5")
model_left=tf.keras.models.load_model("LEFT_MODEL.h5")
model_right=tf.keras.models.load_model("RIGHT_MODEL.h5")
model_whole=tf.keras.models.load_model("full_image_model.h5")

In [49]:
feature_top_test=model_header.predict(X_test)
feature_bottom_test=model_footer.predict(X_test)
feature_left_test=model_left.predict(X_test)
feature_right_test=model_right.predict(X_test)
feature_whole_test=model_whole.predict(X_test)

29/29 [==============================] - 4s 121ms/step


In [50]:
total_features_test=np.hstack((feature_top_test,feature_bottom_test,feature_left_test,feature_right_test,feature_whole_test))

In [51]:
predictions=final_model.predict(total_features_test)

29/29 [==============================] - 0s 4ms/step


In [54]:
y_classes_1 = [np.argmax(element) for element in predictions]
y_classes_1[:20]

[4, 6, 4, 8, 3, 8, 10, 3, 2, 4, 0, 15, 5, 12, 14, 9, 13, 1, 9, 12]

In [55]:
y_classes_2 = [np.argmax(element) for element in feature_left_test]
y_classes_2[:20]

[4, 6, 4, 8, 3, 8, 12, 3, 2, 4, 7, 15, 12, 12, 14, 9, 13, 11, 9, 12]

In [56]:
df3 = pd.DataFrame(y_classes_1,columns = ['label'])
result_1 = pd.concat([df, df3], axis=1)
result_1.set_index('id',inplace = True)
result_1.to_csv("C:/Users/nithi/.jupyter/new_final.csv")

In [57]:
result_1

,label
id,
17801,4
17802,6
17803,4
17804,8
17805,3
...,...
18696,1
18697,12
18698,8
